In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K

Using TensorFlow backend.


In [2]:
gc.enable()

In [3]:
steps = 5
height = 108
width = 108

In [4]:
def main(optimizer, loss):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [16]:
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    wrong = 0
    for i in range(prediction.shape[0]):
        for j in range(prediction.shape[1]):
            if abs(prediction[i][j] - y_test[i][j]) > 0.5:
                wrong += 1
                break
    print(prediction.shape[0])
    print(wrong)
    accuracy = 1 - wrong/(prediction.shape[0])
    return accuracy

In [7]:
classifier = main(optimizer='adam', loss='binary_crossentropy')
classifier.load_weights('weight-spectrums2-all-lr0.01.hdf5')

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


In [9]:
#This is an example

In [8]:
#change the path to where you save the input numpy arrays
lstm = np.array(glob.glob('E:/notes_database/lstm_inputs_2/*'))
cnn = np.array(glob.glob('E:/notes_database/cnn_inputs_2/*'))
y_train = np.array(glob.glob('E:/notes_database/y_train_2/*'))

In [21]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [10]:
def generate_test_arrays(test_order, num=0):
    cnn_test = np.load(cnn[test_order[num][0]])
    labels_test = np.load(y_train[test_order[num][0]])
    lstm_test = np.load(lstm[test_order[num][0]])
    
    count = 0
    ce = 0
    for i in test_order[num][1:]:
        if count == ce + 50:
            print(count)
            ce = count
        count+=1
        
        lstm_test = np.append(lstm_test, np.load(lstm[i]), axis=0)
        cnn_test = np.append(cnn_test, np.load(cnn[i]), axis=0)
        labels_test = np.append(labels_test, np.load(y_train[i]), axis=0)
    
    return cnn_test, lstm_test, labels_test

In [22]:
a,b,c = generate_test_arrays(test_order)

50


In [23]:
accuracy = load_test_score(classifier, a, b, c)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
accuracy

In [20]:
# If you wanna see the output in the standard format
def return_prediction(prediction):
    output = 'The notes are: C4'
    for i in range(prediction.shape[0]):
        for j in range(prediction.shape[1]):
            if prediction[i][j] < 0.5:
                prediction[i][j] = 0
            elif prediction[i][j] > 0.5:
                prediction[i][j] = 1
                output+='_{}'.format(j+60)
        print(output)
    return prediction